In [42]:
import math
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import numpy as np
import re

%load_ext autoreload
%autoreload 2

cookies = {
        'NGSESSID': '861c9e7186cb65420418f0a3ff495c61',
    }

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'cs,sk;q=0.8,en-US;q=0.5,en;q=0.3',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.connexservice.com/',
    'X-Requested-With': 'XMLHttpRequest',
    'DNT': '1',
    'Connection': 'keep-alive',
    # 'Cookie': 'NGSESSID=861c9e7186cb65420418f0a3ff495c61',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
with requests.session() as session:
    response = requests.get(
        'https://www.connexservice.com/HolidayPlus_Preview/search?filters=%7B%22travel_period%22:%7B%22from%22:null,%22to%22:null%7D,%22travel_themes%22:[],%22fittings%22:[],%22positions%22:[],%22countries%22:null,%22regions%22:null%7D&paging=%7B%22limit%22:10,%22offset%22:0%7D',
        cookies=cookies,
        headers=headers,
    )

    with open('hotel.html', 'wb+') as f:
        f.write(response.content)



In [26]:
data = json.loads(response.content)
hotel_ids = []
for hotel in data['results']['partners']['result']:
    # print(hotel['doc']['prm_partner_id'])
    hotel_ids.append([hotel['doc']['name'], hotel['doc']['country_message_key'], hotel['doc']['country_region_message_key'], hotel['doc']['prm_partner_id']])



In [185]:
temp = data['results']['partners']['result']
print(temp)


[{'doc': {'prm_partner_id': '2004', 'city': 'Prag/Praha', 'hotel_id': 'H0024907', 'country_message_key': 'Česká republika', 'partner_number': 'H0024907', 'region_id': '423', 'travel_themes': ['38'], 'language': '89', 'positions': [], 'logo_src': '', 'img_src': 'https://imgs.connexservice.com/partner_img/H0026000/H0024907/H0024907_ag.jpg', 'zipcode': '10000', 'classifications': [{'cnt_total': '12', 'max_classification': '5', 'id': '6', 'categories': ['438', '449', '455', '471', '500'], 'classification': '2', 'cnt_matches': '5'}], 'phone': '00420 274 777 717', 'street': 'Okrajní 1', 'name': 'Hotel Villa', 'fittings': [], 'currency': '1', 'location': {'lon': 14.48241, 'lat': 50.081577}, 'fax': '00420 274 777 719', 'country_region_message_key': 'Praha', 'email': 'reservation@hotel-villa.cz', 'country_iso': 'CZ', 'homepage': 'http://www.hotel-villa.cz', 'text_short': 'V klidném místě vilové čtvrti Prahy, blízko stanice metra Želivského, Vás očekává hotel Villa, který je obklopen překrásnou 

In [190]:
for i, hotel in enumerate(hotel_ids):
    # print(i, hotel[3], temp[i]['doc']['prm_partner_id'])
    if len(hotel) > 6:
        continue
    for j in temp:
        if hotel[3] == j['doc']['prm_partner_id']:
            # print(j)
            if 'location' in j['doc'].keys():
                hotel.append(j['doc']['location'])
            else:
                print('no location for')
                print(hotel)
            hotel.append(j['doc']['homepage'])
            break
    else:
        print(hotel)

In [44]:
with requests.session() as session:
    for hotel in hotel_ids:
        if len(hotel) == 4:
            try:
                response1 = requests.get('https://www.connexservice.com/HolidayPlus_Preview/hoteldetail?partner_id='+hotel[3], cookies=cookies, headers=headers,)
                # with open('hotel1.html', 'wb+') as f:
                #     f.write(response1.content)
                soup = BeautifulSoup(response1.content, "html.parser")
                bar = soup.find("div", {"class": "col-xs-12 padding-0-r padding-0-l"})
                text = re.sub('ww+', '', bar.text)
                text = re.sub(r'\s{2,}', '\n', text)
                hotel.append(text)
            except Exception as e:
                print(hotel)
                print(e)

        # break


'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'


In [43]:
with open('hotel.html', 'wb+') as f:
    f.write(response1.content)


In [48]:
for hotel in hotel_ids:
    if len(hotel) == 5:
        print(hotel[:4])
        print(hotel[4])
        # break


['Hotel Villa', 'Česká republika', 'Praha', '2004']

Pobyt s TRAVELCARD unlimited
:
1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby
Povinně se hradí snídaně a večeře na osobu a noc.
Cena za snídani a večeři na osobu a noc:
EUR 48,00
na osobu
za noc
Večeře se skládá z 3chodového à la carte menu

['Hotel Ametyst', 'Česká republika', 'Praha', '553']

Pobyt s TRAVELCARD unlimited
:
1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby
Povinně se hradí snídaně a večeře na osobu a noc.
Cena za snídani a večeři na osobu a noc:
01.01. - 31.03.:
EUR 50,00
na osobu
za noc
Ubytování ve dvoulůžkovém pokoji Deluxe
01.04. - 31.10.:
EUR 60,00
na osobu
za noc
Ubytování ve dvoulůžkovém pokoji Deluxe
01.11. - 31.12.:
EUR 50,00
na osobu
za noc
Ubytování ve dvoulůžkovém pokoji Deluxe

['Le Palais Art Hotel Prague', 'Česká republika', 'Praha', '25241']

Pobyt s TRAVELCARD unlimited
:
1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby
Povinně se hradí snídaně a večeře na osobu a noc.
Cena za snídani a večeři na osobu a no

In [165]:
for hotel in hotel_ids:
    if len(hotel) == 5:
        print(hotel)
        cost = price(hotel[4])
        if cost == 0:
            print('PANIK')
        hotel.append(cost)

    elif len(hotel) > 5:
        print(hotel)
        cost = price(hotel[4])
        if cost == 0:
            print('PANIK')
        hotel[5] = cost


['Hotel Altora', 'Německo', 'Sachsen-Anhalt (Sasko - Anhaltsko)', '2476', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa012,00\nna osobu\nza noc\nCena večeře á la carte na osobu a noc:\nEUR\xa030,00\nna osobu\nza noc\n', 12.0]
['Hotel Antica Marina', 'Itálie', 'Kalábrie', '3257', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa012,00\nna osobu\nza noc\nCena večeře na osobu/noc (v zimě):\nEUR\xa032,00\nna osobu\nza noc\n', 12.0]
['The Preston International Hotel', 'Velká Británie', 'North West', '1937', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nGBP\xa012,95\nna osobu\nza noc\nCena povinné minimální konzumace (snídaně

In [192]:
for hotel in hotel_ids:
    if len(hotel) != 8:
        print(hotel)
        hotel_ids.remove(hotel)

['Brook Red Lion Hotel', 'Velká Británie', 'East Anglia', '30241', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídani a večeři na osobu a noc:\nGBP\xa044,00\nna osobu\nza noc\nVečeře se skládá z 3-chodového menu.\n', 88.0, 'http://www.brook-hotels.co.uk']
['Six Bells Inn', 'Velká Británie', 'East Anglia', '1833', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídani a večeři na osobu a noc:\nGBP\xa045,00\nna osobu\nza noc\n', 90.0, 'http://www.sixbellsbardwell.co.uk']
['Manor of Groves Hotel, Golf & Country Club', 'Velká Británie', 'Hertfordshire', '31685', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídani a večeři na osobu a noc:\nGBP\xa056,00\nna osobu\nza noc\n', 112.0, 'https://www.manorofgro

In [166]:
hotel_ids.sort(key=lambda x: x[5])
for hotel in hotel_ids:
    print(hotel)
    # print(hotel[0:4]+hotel[5:])


['Hotel Bravo', 'Česká republika', 'Pardubický kraj', '383', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa05,00\nna osobu\nza noc\nCena za 3-chodové menu na osobu a noc:\nEUR\xa012,00\nna osobu\nza noc\n', 17.0]
['Csipke Hotel', 'Maďarsko', 'Bács-Kiskun', '32679', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa07,00\nna osobu\nza noc\nCena za 3-chodové menu na osobu a noc:\nEUR\xa010,00\nna osobu\nza noc\n', 17.0]
['Hotel Silvanus', 'Maďarsko', 'Pest', '2117', '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa018,00\nna osobu\nza noc\nCena večerního bufetu na osobu a noc (v zimě):\nEUR\xa033,00\nna osobu\nza noc\n',

In [164]:
text = '\nPobyt s TRAVELCARD unlimited\n:\n1-6 noc(i) ve dvoulůžkovém pokoji pro 2 osoby\nPovinně se hradí snídaně a večeře na osobu a noc.\nCena za snídaní na osobu a noc:\nEUR\xa08,50\nna osobu\nza noc\nRate per person and night for the à la carte menu\nEUR\xa025,00\nna osobu\nza noc\n'

# text = hotel_ids[69][4]
# print(text)


def price(text):
    lines = text.split("\n")

    summer_breakfast_price = 0
    winter_breakfast_price = 0
    summer_menu_price = 0
    winter_menu_price = 0

    currencies = ["EUR", "GBP", "CHF"]

    for i, line in enumerate(lines):
        for currency in currencies:
            brk = True
            if "Cena za snídaní " in line or " za snídani " in line or "breakfast and dinner" in line and i+2 < len(lines):
                if "léto: "+currency in lines[i+1]:
                    summer_breakfast_price = float(lines[i+1].split(" ")[2].replace(",", "."))
                elif "zima: "+currency in lines[i+2]:
                    winter_breakfast_price = float(lines[i+2].split(" ")[2].replace(",", "."))
                elif currency in lines[i+1]:
                    # print(lines[i+1].split("\xa0"))
                    winter_breakfast_price = float(lines[i+1].split("\xa0")[1].replace(",", "."))
                elif currency in lines[i+2]:
                    # print(lines[i+1].split("\xa0"))
                    winter_breakfast_price = float(lines[i+2].split("\xa0")[1].replace(",", "."))
                else:
                    brk = False
            if ((("menu" in line or "večeř" in line) and ":" in line) or "per person and night for" in line) and i+2 < len(lines):
                if "léto: "+currency in lines[i+1]:
                    summer_menu_price = float(lines[i+1].split(" ")[2].replace(",", "."))
                elif "zima: "+currency in lines[i+2]:
                    winter_menu_price = float(lines[i+2].split(" ")[2].replace(",", "."))
                elif currency in lines[i+1]:
                    # print(lines[i+1].split("\xa0"))
                    winter_menu_price = float(lines[i+1].split("\xa0")[1].replace(",", "."))
                elif currency in lines[i+2]:
                    # print(lines[i+1].split("\xa0"))
                    winter_menu_price = float(lines[i+2].split("\xa0")[1].replace(",", "."))
                else:
                    brk = False
            if brk:
                break

    min_summer_price = summer_breakfast_price + summer_menu_price
    min_winter_price = winter_breakfast_price + winter_menu_price

    # print("Summer:", min_summer_price)
    # print("Winter:", min_winter_price)

    return min(min_summer_price, min_winter_price) if min_summer_price != 0 and min_winter_price != 0 else max(min_summer_price, min_winter_price)

print(price(text))


33.5


In [198]:
import folium

# Create a map centered on the mean latitude and longitude of the data points
mean_lat = sum(location[6]['lat'] for location in hotel_ids) / len(hotel_ids)
mean_lon = sum(location[6]['lon'] for location in hotel_ids) / len(hotel_ids)
map = folium.Map(location=[mean_lat, mean_lon], zoom_start=13)

# Add a marker for each data point
for location in hotel_ids:
    l = location[6]
    pop = ''
    for s in location:
        pop += str(s) + '\n'
    folium.Marker(
        location=[l['lat'], l['lon']],
        popup=pop
    ).add_to(map)
    # break

# Save the map to an HTML file
map.save('map.html')
